In [2]:
install.packages("caret")
install.packages("randomForest")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’




In [1]:
# Load necessary libraries
library(dplyr)
library(tidyr)
library(readr)
library(caret)
library(randomForest)
library(ggplot2)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: ggplot2

Loading required package: lattice

randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin


The following object is masked from ‘package:dplyr’:

    combine




In [2]:
# Load datasets
case_data <- read_csv('PC_case.csv', col_names = FALSE)
control_data <- read_csv('PC_control.csv', col_names = FALSE)
gene_data <- read_csv('geneList.csv', col_names = FALSE)

Rows: 11564 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): X1, X4, X5
dbl (2): X2, X3

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 32449 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): X1, X4, X5
dbl (2): X2, X3

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 59110 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): X1, X3
dbl (3): X2, X4, X5

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
# Rename columns for clarity
colnames(case_data) <- c('Chromosome', 'Start', 'End', 'Type', 'Patient_ID')
colnames(control_data) <- c('Chromosome', 'Start', 'End', 'Type', 'Patient_ID')
colnames(gene_data) <- c('Gene_ID', 'Chromosome_ID', 'Chromosome', 'Gene_Start', 'Gene_End')

In [4]:
# Combine case and control data
case_data <- mutate(case_data, target = 1)
control_data <- mutate(control_data, target = 0)
combined_data <- bind_rows(case_data, control_data)

In [5]:
# Map significant regions to genes
map_to_genes <- function(chromosome, start, end, genes) {
  overlapping_genes <- genes %>%
    filter(Chromosome == paste0("chr", chromosome),
           Gene_End >= start,
           Gene_Start <= end)
  paste(overlapping_genes$Gene_ID, collapse = ";")
}

combined_data <- combined_data %>%
  rowwise() %>%
  mutate(Associated_Genes = map_to_genes(Chromosome, Start, End, gene_data))

In [6]:
# Export the combined data to a CSV file
write_csv(combined_data, "combined_data.csv")

In [7]:
# Feature engineering
combined_data <- combined_data %>%
  mutate(VariationLength = End - Start) %>%
  select(-Patient_ID)

In [ ]:
# One-Hot Encoding
encoded_data <- combined_data %>%
  mutate(across(c(Chromosome, Type, Associated_Genes), as.factor)) %>%
  model.matrix(~ . - 1, data = .) %>%
  as.data.frame()

In [ ]:
# Train a model
set.seed(42)
X <- encoded_data %>% select(-target)
y <- encoded_data$target
train_index <- createDataPartition(y, p = 0.8, list = FALSE)
X_train <- X[train_index, ]
X_test <- X[-train_index, ]
y_train <- y[train_index]
y_test <- y[-train_index]

model <- randomForest(X_train, as.factor(y_train))

In [ ]:
# Evaluate the model
y_pred <- predict(model, X_test)
confusion_matrix <- confusionMatrix(as.factor(y_pred), as.factor(y_test))
print(confusion_matrix)

In [ ]:
# Feature importance
feature_importance <- data.frame(Feature = colnames(X_train), Importance = importance(model)) %>%
  arrange(desc(Importance))

In [ ]:
# Identify significant genes
gene_columns <- grep("Associated_Genes_", colnames(encoded_data), value = TRUE)
significant_genes <- encoded_data %>%
  filter(target == 1) %>%
  select(all_of(gene_columns)) %>%
  summarise(across(everything(), sum)) %>%
  pivot_longer(everything(), names_to = "Gene", values_to = "Count") %>%
  arrange(desc(Count))

write_csv(significant_genes, "significant_genes.csv")

In [ ]:
# Gene counts
significant_genes <- read_csv("significant_genes.csv")

In [ ]:
# Split the genes and count occurrences
gene_counts <- significant_genes %>%
  separate_rows(Gene, sep = ";") %>%
  count(Gene, name = "Count") %>%
  arrange(desc(Count))

write_csv(gene_counts, "gene_counts.csv")

In [ ]:
# Plot the top 20 genes with the highest counts
top_genes <- head(gene_counts, 20)

ggplot(top_genes, aes(x = reorder(Gene, Count), y = Count)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  coord_flip() +
  labs(title = "Top 20 Genes with Highest Counts", x = "Gene", y = "Count") +
  theme_minimal()